In [3]:

# Standard imports
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display


audio_path = r"C:\Users\17205\Music\anti-alexa.wav"
audio_data, sample_rate = librosa.load(audio_path, duration=120)


spec_mag, phase = librosa.magphase(librosa.stft(audio_data))


time_slice = slice(*librosa.time_to_frames([0, 5], sr=sample_rate))
plt.figure(figsize=(12, 4))
librosa.display.specshow(librosa.amplitude_to_db(spec_mag[:, time_slice], ref=np.max),
                         y_axis='log', x_axis='time', sr=sample_rate)
plt.colorbar()
plt.tight_layout()



filtered_spec = librosa.decompose.nn_filter(spec_mag,
                                           aggregate=np.median,
                                           metric='cosine',
                                           width=int(librosa.time_to_frames(2, sr=sample_rate)))


filtered_spec = np.minimum(spec_mag, filtered_spec)


margin_inst, margin_voc = 2, 10
power = 2

mask_inst = librosa.util.softmask(filtered_spec,
                                  margin_inst * (spec_mag - filtered_spec),
                                  power=power)

mask_voc = librosa.util.softmask(spec_mag - filtered_spec,
                                 margin_voc * filtered_spec,
                                 power=power)


foreground = mask_voc * spec_mag
background = mask_inst * spec_mag



plt.figure(figsize=(12, 8))
plt.subplot(3, 1, 1)
librosa.display.specshow(librosa.amplitude_to_db(spec_mag[:, time_slice], ref=np.max),
                         y_axis='log', sr=sample_rate)
plt.title('Full spectrum')
plt.colorbar()

plt.subplot(3, 1, 2)
librosa.display.specshow(librosa.amplitude_to_db(background[:, time_slice], ref=np.max),
                         y_axis='log', sr=sample_rate)
plt.title('Background')
plt.colorbar()

plt.subplot(3, 1, 3)
librosa.display.specshow(librosa.amplitude_to_db(foreground[:, time_slice], ref=np.max),
                         y_axis='log', x_axis='time', sr=sample_rate)
plt.title('Foreground')
plt.colorbar()
plt.tight_layout()
plt.savefig('Signal_Background_Foreground_Separated.jpg')
plt.show()


C:\Users\17205\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


SystemError: initialization of _internal failed without raising an exception